In [7]:
import numpy as np
import random
from PIL import Image

def to_binary(pixel_values):
    return "".join(format(val, "08b") for val in pixel_values)

def to_decimal(binary_str):
    return [int(binary_str[i:i + 8], 2) for i in range(0, len(binary_str), 8)]

def modinv(a, m):
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1

class KnapsackCryptosystem:
    def __init__(self, length=8):
        self.private_key = self.generate_private_key(length)
        self.m = sum(self.private_key) + random.randint(10, 20)
        self.n = random.randint(2, self.m - 1)
        while np.gcd(self.n, self.m) != 1:
            self.n = random.randint(2, self.m - 1)
        self.public_key = [(self.n * pk_elem) % self.m for pk_elem in self.private_key]

    def generate_private_key(self, length):
        private_key = [random.randint(1, 10)]
        for _ in range(1, length):
            next_value = sum(private_key) + random.randint(1, 10)
            private_key.append(next_value)
        return private_key

    def encrypt(self, pixel_values):
        binary_str = to_binary(pixel_values)
        cipher_blocks = []
        for i in range(0, len(binary_str), 8):
            chunk = binary_str[i: i + 8].ljust(8, "0")
            encrypted_sum = sum(int(bit) * self.public_key[j] for j, bit in enumerate(chunk))
            cipher_blocks.append(encrypted_sum)
        return cipher_blocks

    def decrypt(self, ciphertext):
        n_inv = modinv(self.n, self.m)
        decrypted_bits = []
        for cipher_block in ciphertext:
            c_prime = (cipher_block * n_inv) % self.m
            bits = ["0"] * len(self.private_key)
            for i in reversed(range(len(self.private_key))):
                if self.private_key[i] <= c_prime:
                    bits[i] = "1"
                    c_prime -= self.private_key[i]
            decrypted_bits.append("".join(bits))
        decrypted_binary_str = "".join(decrypted_bits)
        return to_decimal(decrypted_binary_str)

def image_to_pixels(image_path):
    image = Image.open(image_path)
    pixel_values = list(image.getdata())
    flat_pixel_values = [val for sublist in pixel_values for val in sublist]
    return flat_pixel_values, image.size, image.mode

def pixels_to_image(pixel_values, image_size, mode):
    reshaped_pixels = [tuple(pixel_values[i:i + 3]) for i in range(0, len(pixel_values), 3)]
    image = Image.new(mode, image_size)
    image.putdata(reshaped_pixels)
    return image

def encrypted_pixels_to_image(encrypted_values, image_size):
    max_value = max(encrypted_values)
    min_value = min(encrypted_values)
    normalized_values = [int((val - min_value) / (max_value - min_value) * 255) for val in encrypted_values]
    return pixels_to_image(normalized_values, image_size, "RGB")

knapsack = KnapsackCryptosystem()
image_path = "images.jpeg"
pixel_values, image_size, mode = image_to_pixels(image_path)
ciphertext = knapsack.encrypt(pixel_values)
encrypted_image = encrypted_pixels_to_image(ciphertext, image_size)
encrypted_image.show()
encrypted_image.save("encrypted_image.png")
decrypted_pixels = knapsack.decrypt
